<a href="https://colab.research.google.com/github/Juliadambros/Aplicacoes-Inteligencia-Artificial/blob/main/Implementa%C3%A7%C3%A3o_de_uma_Rede_Neural_Multicamadas_com_Backpropagation_para_Autentica%C3%A7%C3%A3o_de_C%C3%A9dulas_Banc%C3%A1rias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação das Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(42)

Funções de ativação

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv_from_activation(a):
    return a * (1 - a)

def init_weights_xavier(layers):
    # layers: ex. [4, 8, 1]
    Ws = []
    for i in range(len(layers) - 1):
        fan_in = layers[i]
        fan_out = layers[i+1]
        # +1 no fan_in por conta do bias
        limit = np.sqrt(6 / (fan_in + fan_out))
        W = np.random.uniform(-limit, limit, size=(fan_in + 1, fan_out))  # inclui bias
        Ws.append(W)
    return Ws


Funções da Rede Neural (MLP)

In [ ]:
def forward(X, Ws):
    """
    X: (n, d) sem a coluna de bias
    Ws: lista de matrizes de pesos (cada uma já com linha de bias)
    retorna: lista de ativações por camada (inclui entrada e saídas intermediárias)
    """
    activations = [X]
    A = X
    for W in Ws:
        A_bias = np.concatenate([A, np.ones((A.shape[0], 1))], axis=1)  # adiciona bias
        Z = A_bias @ W
        A = sigmoid(Z)
        activations.append(A)
    return activations

def backprop(activations, y, Ws, lr=0.1, l2=0.0, loss="bce"):
    """
    BCE com saída sigmoide: delta_out = (A_L - y)
    activations: lista de ativações de cada camada (vinda do forward).
    y: rótulos reais.
    Ws: lista de pesos (será atualizada).
    lr: taxa de aprendizado.
    l2: regularização L2 (penaliza pesos grandes).
    loss: função de erro (padrão = Binary Cross Entropy).
    """
    y = y.reshape(-1, 1)
    A_L = activations[-1]
    if loss == "bce":
        delta = (A_L - y)  # derivada simplifica com sigmoide
    else:
        delta = (A_L - y) * sigmoid_deriv_from_activation(A_L)

    deltas = [delta]
    # camadas ocultas (percorrer Ws de trás p/ frente, exceto a última)
    for i in range(len(Ws)-2, -1, -1):
        W_next = Ws[i+1][:-1, :]  # remove linha do bias
        A_i = activations[i+1]    # ativação da camada i+1 (já pós-sigmoide)
        delta = (deltas[0] @ W_next.T) * sigmoid_deriv_from_activation(A_i)
        deltas.insert(0, delta)

    # atualiza pesos
    for i in range(len(Ws)):
        A_i = activations[i]
        A_i_bias = np.concatenate([A_i, np.ones((A_i.shape[0], 1))], axis=1)
        grad = A_i_bias.T @ deltas[i] / A_i.shape[0]
        if l2 > 0:
            # L2 só nos pesos (não no bias): zera a última linha da penalização
            pen = np.copy(Ws[i])
            pen[-1, :] = 0.0
            grad += l2 * pen
        Ws[i] -= lr * grad
    return Ws

def binary_cross_entropy(y_true, y_prob, eps=1e-12): #função de perda
    y_true = y_true.reshape(-1, 1)
    y_prob = np.clip(y_prob, eps, 1 - eps)
    return np.mean(- (y_true * np.log(y_prob) + (1 - y_true) * np.log(1 - y_prob))) #retorna um número pequeno se a rede acertou (boa previsão) e grande se errou muito.

def predict(X, Ws, thr=0.5):
    A = forward(X, Ws)[-1]
    return (A >= thr).astype(int) #A >= 0.5, retorna 1 e se A < 0.5, retorna 0.

def accuracy(y_true, y_pred):
    y_true = y_true.reshape(-1, 1)
    return (y_true == y_pred).mean() * 100


Carregar e preparar o dataset

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
cols = ["variance", "skewness", "curtosis", "entropy", "class"]
df = pd.read_csv(url, header=None, names=cols)

X = df.iloc[:, :-1].values.astype(float) # todas as colunas, exceto a última
y = df.iloc[:, -1].values.astype(int) # apenas a última coluna (classe)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# padronização usando média e desvio do treino
mu = X_tr.mean(axis=0)
sigma = X_tr.std(axis=0, ddof=0)
sigma[sigma == 0] = 1.0  # evita divisão por zero

X_tr = (X_tr - mu) / sigma
X_te = (X_te - mu) / sigma

print("Treino:", X_tr.shape, "Teste:", X_te.shape, "Positivos em teste:", y_te.sum())


Treino: (960, 4) Teste: (412, 4) Positivos em teste: 183


Definir parâmetros e treinar

In [ ]:
layers = [X_tr.shape[1],8,8,1]
#layers = [4, 10, 8, 6, 1]
# 4 entradas → oculta(10) → oculta(8) → oculta(6) → 1 saída

lr = 1 # taxa de aprendizado
epochs = 500
l2 = 0.000 #penaliza pesos grandes

Ws = init_weights_xavier(layers)
history = []

for ep in range(1, epochs + 1):
    acts = forward(X_tr, Ws)
    Ws = backprop(acts, y_tr, Ws, lr=lr, l2=l2, loss="bce")
    if ep % 50 == 0 or ep == 1:
        loss = binary_cross_entropy(y_tr, acts[-1])
        history.append((ep, loss))

print("Treino finalizado.")
for ep, loss in history:
    print(f"Época {ep:4d} | Loss treino (BCE): {loss:.4f}")


Treino finalizado.
Época    1 | Loss treino (BCE): 0.7104
Época   50 | Loss treino (BCE): 0.3193
Época  100 | Loss treino (BCE): 0.0872
Época  150 | Loss treino (BCE): 0.0522
Época  200 | Loss treino (BCE): 0.0399
Época  250 | Loss treino (BCE): 0.0339
Época  300 | Loss treino (BCE): 0.0303
Época  350 | Loss treino (BCE): 0.0280
Época  400 | Loss treino (BCE): 0.0263
Época  450 | Loss treino (BCE): 0.0249
Época  500 | Loss treino (BCE): 0.0234


Avaliar a rede

In [ ]:
y_hat = predict(X_te, Ws, thr=0.5)
acc = accuracy(y_te, y_hat)

print(f"Acurácia no teste: {acc:.2f}%")



Acurácia no teste: 99.03%
